## Metrics Evaluation

In [3]:
import pandas as pd
import os
import sys
import glob

In [2]:
def find_results(base_dir):
    val_dfs = []
    minds_test_dfs = []
    malta_test_dfs = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            # check if file exists and if it ends with validation_results.csv
            if file.startswith('validation_results.csv'):
                val_df = pd.read_csv(os.path.join(root, file))
                val_df['model_name'] = root.split('/')[-1]
                val_dfs.append(val_df)
            elif file.startswith('malta_test'):
                malta_test_df = pd.read_csv(os.path.join(root, file))
                malta_test_df['model_name'] = root.split('/')[-1]
                malta_test_dfs.append(malta_test_df)
            elif file.startswith('minds_test'):
                minds_test_df = pd.read_csv(os.path.join(root, file))
                minds_test_df['model_name'] = root.split('/')[-1]
                minds_test_dfs.append(minds_test_df)
    val_df = pd.concat(val_dfs)
    malta_test_df = pd.concat(malta_test_dfs)
    minds_test_df = pd.concat(minds_test_dfs)
    
    return val_df, minds_test_df, malta_test_df

In [3]:
def group_metrics_by_intensity(df, sort_ascending=True):

    # Extract intensity and seed from experiment name
    df[['intensity', 'seed']] = df['exp_name'].str.extract(r'_(\d+\.\d+)_(\d+)$')
    df['intensity'] = df['intensity'].astype(float)
    df['seed'] = df['seed'].astype(int)
    
    # Filter for overall metrics only
    if 'dictionary' not in df.columns:
        return df
    else:
        overall_df = df[df['dictionary'].str.lower() == 'overall'].copy()
    
    # Group by intensity and aggregate metrics
    grouped = overall_df.groupby('intensity').agg({
        'acc': ['mean', 'std'],
        'top5_acc': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std']
    })
    
    # Flatten multi-index columns
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
    
    # Reset index and sort
    grouped = grouped.reset_index().sort_values('intensity', ascending=sort_ascending)
    
    return grouped

def group_metrics_by_dictionary(df, include_intensity=True, sort_ascending=True):

    # Extract intensity and seed
    df[['intensity', 'seed']] = df['exp_name'].str.extract(r'_(\d+\.\d+)_(\d+)$')
    df['intensity'] = df['intensity'].astype(float)
    df['seed'] = df['seed'].astype(int)
    
    # Exclude overall metrics
    dict_df = df[df['dictionary'].str.lower() != 'overall'].copy()
    
    # Determine grouping columns
    group_cols = ['intensity', 'dictionary'] if include_intensity else ['dictionary']
    
    # Aggregate metrics
    grouped = dict_df.groupby(group_cols).agg({
        'acc': ['mean', 'std'],
        'top5_acc': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std']
    })
    
    # Flatten column names
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
    
    # Sort and reset index
    return grouped.reset_index().sort_values(group_cols, ascending=sort_ascending)

## TimeSformer

#### Pre-trained on Kinetics

In [ ]:
val

## Agressive Splits (40/40/20)

#### No Augs

In [4]:
val_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/base/agressive_splits_videomae_kinetics_no_augmentations_42/agressive_splits_videomae_kinetics_no_augmentations_42_validation_results.csv')

minds_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/base/agressive_splits_videomae_kinetics_no_augmentations_42/minds_test_metrics_results.csv')

malta_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/base/agressive_splits_videomae_kinetics_no_augmentations_42/malta_test_metrics_results.csv')

In [5]:
val_df['model_name'] = val_df['model_name'].str.replace('agressive_splits_videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,no_augmentations_42,0.962121,0.997917,0.962263,0.962121,0.964914


In [6]:
minds_test_df = minds_test_df[minds_test_df['dictionary'] != 'Overall']
minds_test_df = minds_test_df.drop(columns=['model','cm'])
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall
0,no_augmentations_42,test,1.0,1.0,1.0,1.0,1.0


In [7]:
malta_test_df = malta_test_df.drop(columns=['model','cm'])
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall
6,no_augmentations_42,Overall,0.587786,0.80916,0.567429,0.599056,0.59248


## Color Jitter

In [8]:
def group_metrics_by_intensity(df, sort_ascending=True):

    # Extract intensity and seed from experiment name
    df[['intensity', 'seed']] = df['exp_name'].str.extract(r'_(\d+\.\d+)_(\d+)$')
    df['intensity'] = df['intensity'].astype(float)
    df['seed'] = df['seed'].astype(int)
    
    # Filter for overall metrics only
    if 'dictionary' not in df.columns:
        return df
    else:
        overall_df = df[df['dictionary'].str.lower() == 'overall'].copy()
    
    # Group by intensity and aggregate metrics
    grouped = overall_df.groupby('intensity').agg({
        'acc': ['mean', 'std'],
        'top5_acc': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std']
    })
    
    # Flatten multi-index columns
    grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
    
    # Reset index and sort
    grouped = grouped.reset_index().sort_values('intensity', ascending=sort_ascending)
    
    return grouped

In [9]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/agressive_split_color_jitter'
val_df, minds_test_df, malta_test_df = find_results(base_dir)

# sort by model_name column
val_df = val_df.sort_values(by='f1_val', ascending=False)
val_df['model_name'] = val_df['model_name'].str.replace('agressive_splits_videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,color_jitter_0.35_42,0.980114,1.000000,0.979950,0.980114,0.981299
0,color_jitter_0.25_42,0.978409,1.000000,0.978049,0.978409,0.979361
0,color_jitter_0.15_42,0.973295,1.000000,0.973367,0.973295,0.975824
0,color_jitter_0.05_42,0.970833,1.000000,0.971311,0.970833,0.974949
0,color_jitter_0.3_42,0.971212,1.000000,0.971135,0.971212,0.973404
0,color_jitter_0.2_42,0.967045,0.997727,0.967053,0.967045,0.970953
0,color_jitter_0.5_42,0.964583,0.990909,0.964079,0.964583,0.968471
0,color_jitter_0.45_42,0.962311,1.000000,0.962389,0.962311,0.965427
0,color_jitter_0.4_42,0.962311,1.000000,0.961874,0.962311,0.964563
0,color_jitter_0.1_42,0.960227,0.995833,0.959893,0.960227,0.963623


In [11]:
minds_test_df = minds_test_df[minds_test_df['dictionary'] == 'test']
minds_test_df = minds_test_df.drop(columns=['model','cm'])
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
minds_test_df = minds_test_df.sort_values(by='f1', ascending=False)
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,model_name
0,color_jitter_0.4_42,test,0.995708,1.0,0.995826,0.996154,0.995833,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.05_42,test,0.995708,1.0,0.995619,0.996154,0.995455,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.15_42,test,0.995708,1.0,0.995619,0.996154,0.995455,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.45_42,test,0.991416,1.0,0.991288,0.992308,0.991667,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.25_42,test,0.991416,1.0,0.991238,0.992308,0.990909,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.1_42,test,0.987124,1.0,0.987261,0.988690,0.987500,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.3_42,test,0.987124,1.0,0.986890,0.988141,0.986742,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.35_42,test,0.987124,1.0,0.986890,0.988141,0.986742,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.2_42,test,0.982833,1.0,0.982550,0.984295,0.982576,agressive_splits_videomae_kinetics_color_jitte...
0,color_jitter_0.5_42,test,0.974249,1.0,0.973159,0.977152,0.973485,agressive_splits_videomae_kinetics_color_jitte...


In [12]:
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df = malta_test_df.drop(columns=['model','cm'])
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
malta_test_df = malta_test_df.sort_values(by='f1', ascending=False)
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,model_name
6,color_jitter_0.15_42,Overall,0.572519,0.824427,0.556536,0.595068,0.578036,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.05_42,Overall,0.549618,0.770992,0.519396,0.581769,0.533036,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.35_42,Overall,0.549618,0.770992,0.512207,0.559689,0.552837,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.1_42,Overall,0.534351,0.824427,0.507103,0.575871,0.517837,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.5_42,Overall,0.526718,0.725191,0.490093,0.568199,0.523016,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.45_42,Overall,0.503817,0.778626,0.488615,0.573785,0.502639,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.4_42,Overall,0.519084,0.786260,0.487530,0.519755,0.524782,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.3_42,Overall,0.488550,0.770992,0.476513,0.532695,0.520802,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.2_42,Overall,0.503817,0.755725,0.457525,0.487916,0.490317,agressive_splits_videomae_kinetics_color_jitte...
6,color_jitter_0.25_42,Overall,0.488550,0.770992,0.446349,0.471406,0.488532,agressive_splits_videomae_kinetics_color_jitte...


## Random Perspective

In [13]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/agressive_split_random_perspective'

val_df, minds_test_df, malta_test_df = find_results(base_dir)
# sort by f1 column
val_df = val_df.sort_values(by='f1_val', ascending=False)
val_df['model_name'] = val_df['model_name'].str.replace('agressive_splits_videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,random_perspective_0.5_42,0.988826,1.000000,0.988792,0.988826,0.989826
0,random_perspective_0.4_42,0.988636,1.000000,0.988620,0.988636,0.989312
0,random_perspective_0.3_42,0.986932,1.000000,0.986780,0.986932,0.987826
0,random_perspective_0.15_42,0.986364,1.000000,0.986421,0.986364,0.987750
0,random_perspective_0.2_42,0.984280,1.000000,0.983884,0.984280,0.985182
0,random_perspective_0.1_42,0.981818,1.000000,0.981716,0.981818,0.983884
0,random_perspective_0.25_42,0.972917,1.000000,0.973023,0.972917,0.975182
0,random_perspective_0.45_42,0.968182,0.997727,0.968433,0.968182,0.972888
0,random_perspective_0.35_42,0.966477,1.000000,0.966412,0.966477,0.969771
0,random_perspective_0.05_42,0.955492,0.997917,0.955460,0.955492,0.959221


In [14]:
minds_test_df
# remove model, model_name columns
minds_test_df = minds_test_df.drop(columns=['model','model_name'])
minds_test_df = minds_test_df.sort_values(by='f1',ascending=False)

# remove "new_splits_videomae_kinetics_" from every row in exp_name column
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')

# remove all Overall rows from dictionary column
minds_test_df = minds_test_df[minds_test_df['dictionary'] != 'Overall']
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,cm
0,random_perspective_0.3_42,test,1.000000,1.0,1.000000,1.000000,1.000000,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.4_42,test,1.000000,1.0,1.000000,1.000000,1.000000,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.5_42,test,0.991416,1.0,0.991652,0.991987,0.991987,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.15_42,test,0.991416,1.0,0.991304,0.991667,0.991667,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.35_42,test,0.991416,1.0,0.991304,0.991667,0.991667,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.45_42,test,0.987124,1.0,0.986933,0.987442,0.987121,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.1_42,test,0.987124,1.0,0.986923,0.987821,0.987121,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.05_42,test,0.987124,1.0,0.986890,0.988141,0.986742,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.2_42,test,0.982833,1.0,0.982343,0.984295,0.982197,"[[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
0,random_perspective_0.25_42,test,0.969957,1.0,0.970036,0.976961,0.969318,"[[11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [15]:
malta_test_df = malta_test_df.drop(columns=['model','model_name'])
malta_test_df = malta_test_df.sort_values(by='f1',ascending=False)
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,cm
6,random_perspective_0.4_42,Overall,0.526718,0.824427,0.499706,0.518724,0.537282,"[[6, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 2, 0, 0, 0,..."
6,random_perspective_0.05_42,Overall,0.488550,0.732824,0.481730,0.542333,0.517398,"[[7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0,..."
6,random_perspective_0.45_42,Overall,0.473282,0.854962,0.455732,0.564583,0.472123,"[[5, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 1, 0, 0, 0,..."
6,random_perspective_0.15_42,Overall,0.503817,0.786260,0.455314,0.524144,0.509048,"[[7, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,..."
6,random_perspective_0.1_42,Overall,0.458015,0.702290,0.440592,0.528281,0.481725,"[[6, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
6,random_perspective_0.2_42,Overall,0.427481,0.702290,0.430912,0.535076,0.472097,"[[5, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 3, 0, 0, 0,..."
6,random_perspective_0.35_42,Overall,0.435115,0.740458,0.425989,0.522386,0.450689,"[[5, 0, 0, 0, 3, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0,..."
6,random_perspective_0.5_42,Overall,0.458015,0.748092,0.422688,0.472937,0.468175,"[[6, 0, 2, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0,..."
6,random_perspective_0.3_42,Overall,0.435115,0.740458,0.403225,0.475803,0.445873,"[[5, 0, 4, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0,..."
6,random_perspective_0.25_42,Overall,0.427481,0.702290,0.398939,0.591637,0.447540,"[[6, 0, 0, 0, 1, 2, 0, 1, 1, 2, 0, 0, 0, 0, 0,..."


## Random Rotation

In [26]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/agressive_split_random_rotation'

val_df, minds_test_df, malta_test_df = find_results(base_dir)
# sort by f1 column
val_df = val_df.sort_values(by='f1_val', ascending=False)
val_df['model_name'] = val_df['model_name'].str.replace('videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,random_rotation_15_42,0.986742,1.000000,0.986640,0.986742,0.987493
0,random_rotation_20_42,0.984470,0.997727,0.984254,0.984470,0.985960
0,random_rotation_25_42,0.984280,1.000000,0.984117,0.984280,0.985509
0,random_rotation_50_42,0.981818,1.000000,0.981667,0.981818,0.984615
0,random_rotation_10_42,0.979924,1.000000,0.979977,0.979924,0.982295
0,random_rotation_45_42,0.977651,1.000000,0.977785,0.977651,0.980135
0,random_rotation_30_42,0.977273,1.000000,0.977441,0.977273,0.979752
0,random_rotation_0.45_42,0.975568,1.000000,0.975179,0.975568,0.978052
0,random_rotation_40_42,0.972917,1.000000,0.973287,0.972917,0.976896
0,random_rotation_5_42,0.972917,1.000000,0.973135,0.972917,0.977590


In [27]:
minds_test_df = minds_test_df.drop(columns=['model','model_name','cm'])
minds_test_df = minds_test_df.sort_values(by='f1',ascending=False)
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('videomae_kinetics_','')
minds_test_df = minds_test_df[minds_test_df['dictionary'] != 'Overall']
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall
0,random_rotation_40_42,test,1.000000,1.000000,1.000000,1.000000,1.000000
0,random_rotation_20_42,test,0.995708,1.000000,0.995652,0.995833,0.995833
0,random_rotation_0.3_42,test,0.995708,1.000000,0.995619,0.996154,0.995455
0,random_rotation_50_42,test,0.995708,1.000000,0.995445,0.995833,0.995455
0,random_rotation_15_42,test,0.991416,1.000000,0.991445,0.992308,0.991288
0,random_rotation_0.35_42,test,0.991416,1.000000,0.991445,0.992308,0.991288
0,random_rotation_25_42,test,0.991416,1.000000,0.991238,0.992308,0.990909
0,random_rotation_10_42,test,0.987124,1.000000,0.987271,0.988141,0.987442
0,random_rotation_45_42,test,0.987124,1.000000,0.987097,0.988141,0.987121
0,random_rotation_0.05_42,test,0.987124,1.000000,0.986980,0.989011,0.986742


In [18]:
malta_test_df = malta_test_df.drop(columns=['model','model_name'])
malta_test_df = malta_test_df.sort_values(by='f1',ascending=False)
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('videomae_kinetics_','')
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,cm
6,random_rotation_35_42,Overall,0.595420,0.755725,0.563583,0.593864,0.583909,"[[8, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0,..."
6,random_rotation_0.3_42,Overall,0.557252,0.786260,0.541456,0.597697,0.568393,"[[6, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,..."
6,random_rotation_10_42,Overall,0.541985,0.793893,0.513013,0.544058,0.547103,"[[7, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 2, 0, 0, 0,..."
6,random_rotation_30_42,Overall,0.549618,0.816794,0.504766,0.599255,0.535754,"[[8, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0,..."
6,random_rotation_0.4_42,Overall,0.488550,0.755725,0.490787,0.570479,0.508981,"[[6, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 1, 0, 0,..."
6,random_rotation_50_42,Overall,0.496183,0.801527,0.478654,0.505306,0.501091,"[[6, 0, 0, 0, 2, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0,..."
6,random_rotation_45_42,Overall,0.503817,0.755725,0.467702,0.542738,0.511706,"[[8, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0,..."
6,random_rotation_20_42,Overall,0.480916,0.763359,0.465235,0.571834,0.508417,"[[7, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,..."
6,random_rotation_0.35_42,Overall,0.488550,0.748092,0.452527,0.505432,0.504921,"[[5, 0, 1, 0, 1, 0, 0, 2, 1, 1, 0, 2, 0, 0, 0,..."
6,random_rotation_0.15_42,Overall,0.488550,0.801527,0.447099,0.487949,0.485377,"[[5, 0, 1, 0, 3, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0,..."


## Mixup

In [21]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/agressive_split_mixup'

val_df, minds_test_df, malta_test_df = find_results(base_dir)
# sort by f1 column
val_df = val_df.sort_values(by='f1_val', ascending=False)
val_df['model_name'] = val_df['model_name'].str.replace('agressive_splits_videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,mixup_0.5_42,0.977273,1.000000,0.977382,0.977273,0.980741
0,mixup_10_42,0.972917,1.000000,0.973162,0.972917,0.975897
0,mixup_1_42,0.972727,0.997727,0.973031,0.972727,0.977514
0,mixup_15_42,0.973106,1.000000,0.972915,0.973106,0.975967
0,mixup_1.5_42,0.970644,1.000000,0.970571,0.970644,0.975496
0,mixup_40_42,0.968561,1.000000,0.968340,0.968561,0.971003
0,mixup_4.5_42,0.968371,1.000000,0.968219,0.968371,0.972000
0,mixup_3_42,0.966667,0.995644,0.966726,0.966667,0.971578
0,mixup_2.5_42,0.965909,1.000000,0.966219,0.965909,0.969358
0,mixup_5_42,0.965909,0.997727,0.966193,0.965909,0.971623


In [22]:
minds_test_df = minds_test_df.drop(columns=['model','model_name','cm'])
minds_test_df = minds_test_df.sort_values(by='f1',ascending=False)
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('agressive_splits_videomae_kinetics_','')
minds_test_df = minds_test_df[minds_test_df['dictionary'] != 'Overall']
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall
0,mixup_0.5_42,test,1.000000,1.000000,1.000000,1.000000,1.000000
0,mixup_10_42,test,1.000000,1.000000,1.000000,1.000000,1.000000
0,mixup_1_42,test,1.000000,1.000000,1.000000,1.000000,1.000000
0,mixup_40_42,test,1.000000,1.000000,1.000000,1.000000,1.000000
0,mixup_2.5_42,test,0.995708,1.000000,0.995652,0.995833,0.995833
0,mixup_15_42,test,0.995708,1.000000,0.995652,0.995833,0.995833
0,mixup_30_42,test,0.995708,1.000000,0.995619,0.996154,0.995455
0,mixup_2_42,test,0.991416,1.000000,0.991478,0.991987,0.991667
0,mixup_3.5_42,test,0.991416,0.995708,0.991304,0.991667,0.991667
0,mixup_25_42,test,0.991416,1.000000,0.991278,0.991667,0.991288


In [25]:
#malta_test_df = malta_test_df.drop(columns=['model','model_name'])
malta_test_df = malta_test_df.sort_values(by='f1',ascending=False)
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('agressive_splits_','')
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,cm
6,mixup_1_42,Overall,0.511450,0.763359,0.494371,0.582832,0.534524,"[[7, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,..."
6,mixup_40_42,Overall,0.526718,0.755725,0.491270,0.577672,0.518194,"[[7, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0,..."
6,mixup_0.5_42,Overall,0.511450,0.770992,0.490886,0.535750,0.506190,"[[7, 0, 0, 0, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0,..."
6,mixup_2.5_42,Overall,0.496183,0.748092,0.475324,0.564492,0.494643,"[[8, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0,..."
6,mixup_3_42,Overall,0.503817,0.770992,0.468885,0.553595,0.511905,"[[7, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,..."
6,mixup_5_42,Overall,0.480916,0.709924,0.462234,0.527051,0.476726,"[[6, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 1, 1, 0,..."
6,mixup_3.5_42,Overall,0.480916,0.748092,0.448988,0.515670,0.480694,"[[6, 0, 1, 0, 1, 0, 1, 1, 0, 3, 0, 1, 0, 0, 0,..."
6,mixup_4_42,Overall,0.473282,0.740458,0.441177,0.547001,0.476032,"[[6, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0,..."
6,mixup_30_42,Overall,0.488550,0.755725,0.438097,0.541957,0.467480,"[[8, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,..."
6,mixup_10_42,Overall,0.442748,0.679389,0.416558,0.513748,0.428869,"[[8, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 1,..."


## Intensities with highest F1-Score in MINDS VAL split

CJ - 0.35 - .979

RP - 0.5 - .988

RR - 15 - .986

Mixup - 0.5 - .977

## Intensities with highest F1-Score in MINDS TEST split

CJ - 0.40 - .996

RP - 0.30 and 0.40 - 1.0

RR - 40 - 1.0

Mixup - 0.5/10/1/40 - 1.0

## Intensities with highest F1-Score in MALTA-LIBRAS

CJ - 0.15 - .556

RP - 0.40 - .499

RR - 35 - .563

Mixup - 1 - .494

#### All best augs based on F1-Score from testing on MALTA-LIBRAS, using MINDS original split (75/25).

In [30]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/old_split_all_best_F1augs'

val_df, minds_test_df, malta_test_df = find_results(base_dir)
# sort by f1 column
val_df = val_df.sort_values(by='f1_val', ascending=False)
val_df['model_name'] = val_df['model_name'].str.replace('videomae_kinetics_','')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,all_best_F1augs_seed_14_OLD_SPLIT_TESTING,1.000000,1.0,1.000000,1.000000,1.000000
0,all_best_F1augs_seed_16_OLD_SPLIT_TESTING,1.000000,1.0,1.000000,1.000000,1.000000
0,all_best_F1augs_seed_2_OLD_SPLIT_TESTING,0.996667,1.0,0.996552,0.996667,0.996667
0,all_best_F1augs_seed_12_OLD_SPLIT_TESTING,0.996667,1.0,0.996552,0.996667,0.996667
0,all_best_F1augs_seed_42_OLD_SPLIT_TESTING,0.996667,1.0,0.996552,0.996667,0.996667
0,all_best_F1augs_seed_26_OLD_SPLIT_TESTING,0.979048,1.0,0.978898,0.979048,0.981017


In [37]:
minds_test_df = find_results(base_dir)[1]
minds_test_df

minds_test_df = minds_test_df.drop(columns=['model','model_name','cm'])
minds_test_df = minds_test_df.sort_values(by='f1',ascending=False)
minds_test_df['exp_name'] = minds_test_df['exp_name'].str.replace('videomae_kinetics_','')
minds_test_df = minds_test_df[minds_test_df['dictionary'] == 'test']
minds_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall
0,all_best_F1augs_seed_14_OLD_SPLIT_TESTING,test,1.000000,1.0,1.000000,1.000000,1.000000
0,all_best_F1augs_seed_2_OLD_SPLIT_TESTING,test,1.000000,1.0,1.000000,1.000000,1.000000
0,all_best_F1augs_seed_16_OLD_SPLIT_TESTING,test,1.000000,1.0,1.000000,1.000000,1.000000
0,all_best_F1augs_seed_42_OLD_SPLIT_TESTING,test,0.995708,1.0,0.995826,0.996154,0.995833
0,all_best_F1augs_seed_12_OLD_SPLIT_TESTING,test,0.995708,1.0,0.995652,0.995833,0.995833
0,all_best_F1augs_seed_26_OLD_SPLIT_TESTING,test,0.991416,1.0,0.991980,0.992857,0.991987


In [39]:
malta_test_df = find_results(base_dir)[2]
malta_test_df = malta_test_df.drop(columns=['model','model_name'])
malta_test_df = malta_test_df.sort_values(by='f1',ascending=False)
malta_test_df = malta_test_df[malta_test_df['dictionary'] == 'Overall']
malta_test_df['exp_name'] = malta_test_df['exp_name'].str.replace('videomae_kinetics_','')
malta_test_df

,exp_name,dictionary,acc,top5_acc,f1,precision,recall,cm
6,all_best_F1augs_seed_16_OLD_SPLIT_TESTING,Overall,0.541985,0.786260,0.517367,0.560782,0.541250,"[[7, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0,..."
6,all_best_F1augs_seed_14_OLD_SPLIT_TESTING,Overall,0.541985,0.725191,0.514848,0.578270,0.537698,"[[7, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 1, 0, 0, 0,..."
6,all_best_F1augs_seed_12_OLD_SPLIT_TESTING,Overall,0.534351,0.732824,0.491155,0.495612,0.523234,"[[8, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0,..."
6,all_best_F1augs_seed_26_OLD_SPLIT_TESTING,Overall,0.519084,0.725191,0.482207,0.535139,0.513393,"[[7, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0,..."
6,all_best_F1augs_seed_2_OLD_SPLIT_TESTING,Overall,0.503817,0.709924,0.478976,0.567744,0.485873,"[[8, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 1, 0, 0, 0,..."
6,all_best_F1augs_seed_42_OLD_SPLIT_TESTING,Overall,0.503817,0.709924,0.473886,0.553436,0.497877,"[[7, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0,..."


##### Using split 70/15/15 (for MINDS alone), all best augs based on F1-Score (from MALTA-LIBRAS) (cj - 0.40; mu - 0.10; RR (low) - 0.30; RP - 0.50)

In [5]:
minds_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/new_split_all_best_F1augs/new_split_all_best_F1augs_RR_low/minds_test_mean_std_results.csv')

minds_test_df

,dictionary,acc_mean,acc_std,top5_acc_mean,top5_acc_std,f1_mean,f1_std,precision_mean,precision_std,recall_mean,recall_std
0,Overall,0.99779,0.002853,1.0,0.0,0.997771,0.002878,0.998,0.002582,0.997778,0.002869
1,test,0.99779,0.002853,1.0,0.0,0.997771,0.002878,0.998,0.002582,0.997778,0.002869


In [6]:
malta_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/new_split_all_best_F1augs/new_split_all_best_F1augs_RR_low/malta_test_mean_std_results.csv')

malta_test_df

,dictionary,acc_mean,acc_std,top5_acc_mean,top5_acc_std,f1_mean,f1_std,precision_mean,precision_std,recall_mean,recall_std
0,Acessibilidades2,0.230769,0.051282,0.430769,0.067353,0.156161,0.038652,0.140686,0.039030,0.221886,0.048261
1,Acessibilidades3,0.241667,0.032867,0.483333,0.048908,0.183340,0.033422,0.208319,0.036408,0.210331,0.043780
2,Overall,0.283206,0.019857,0.494656,0.043896,0.225691,0.016955,0.237470,0.017168,0.249899,0.018360
3,SpreadTheSign,0.300000,0.039528,0.493750,0.062152,0.227974,0.027363,0.209146,0.024576,0.273424,0.037529
4,UFPE,0.296296,0.042767,0.514815,0.102477,0.157698,0.040849,0.150710,0.056871,0.200626,0.038316
5,UFSC,0.352632,0.043330,0.468421,0.052338,0.309600,0.047134,0.307846,0.053988,0.335819,0.048801
6,UFSC_V2,0.305263,0.048365,0.594737,0.061026,0.264912,0.054671,0.271053,0.058691,0.278947,0.048365


## Split 75/25 on MINDS, test on MALTA-LIBRAS

In [14]:
base_dir = '/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/base'
def find_old_results(base_dir):
    val_dfs = []
    
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            
            if file.startswith('videomae') and file.endswith('validation_results.csv'):
                val_df = pd.read_csv(os.path.join(root, file))
                val_df['model_name'] = root.split('/')[-1]
                val_dfs.append(val_df)
    val_df = pd.concat(val_dfs)
        
    return val_df

No augs with pre-train on Kinetics

In [15]:
val_df = find_old_results(base_dir)
val_df 


,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,videomae_kinetics_no_augmentations_2,1.000000,1.0,1.000000,1.000000,1.000000
0,videomae_kinetics_no_augmentations_16,0.996429,1.0,0.996535,0.996429,0.996875
0,videomae_kinetics_no_augmentations_42,1.000000,1.0,1.000000,1.000000,1.000000
0,videomae_kinetics_no_augmentations_89,0.989524,1.0,0.989609,0.989524,0.990392
0,videomae_kinetics_no_augmentations_12,0.993095,1.0,0.992976,0.993095,0.993333
0,videomae_kinetics_no_augmentations_44,0.996667,1.0,0.996552,0.996667,0.996667
0,videomae_kinetics_no_augmentations_14,0.996429,1.0,0.996535,0.996429,0.996875
0,videomae_kinetics_no_augmentations_26,0.996667,1.0,0.996552,0.996667,0.996667
0,videomae_kinetics_no_augmentations_79,0.982619,1.0,0.982556,0.982619,0.984118
0,videomae_kinetics_no_augmentations_24,0.996667,1.0,0.996552,0.996667,0.996667


In [16]:
malta_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/base/test_mean_std_results.csv')
malta_test_df

,dictionary,acc_mean,acc_std,top5_acc_mean,top5_acc_std,f1_mean,f1_std,precision_mean,precision_std,recall_mean,recall_std
0,Acessibilidades2,0.234615,0.021833,0.496154,0.052705,0.191229,0.032520,0.177693,0.037164,0.248465,0.029725
1,Acessibilidades3,0.300000,0.058267,0.570833,0.062268,0.244436,0.058908,0.262364,0.063668,0.280521,0.056696
2,Overall,0.290840,0.021726,0.541985,0.027405,0.227920,0.022999,0.256779,0.028718,0.258885,0.017410
3,SpreadTheSign,0.293750,0.042184,0.487500,0.049301,0.207146,0.048097,0.179553,0.053311,0.272222,0.042415
4,UFPE,0.314815,0.031475,0.518519,0.081892,0.160495,0.026344,0.150181,0.041795,0.214451,0.029641
5,UFSC,0.331579,0.049931,0.573684,0.057921,0.276326,0.050948,0.266228,0.058041,0.306871,0.049624
6,UFSC_V2,0.278947,0.061026,0.615789,0.070394,0.229561,0.058216,0.233904,0.058091,0.248816,0.057610


No augs, no pre-train on Kinetics

In [17]:
val_df = find_old_results('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/no_pretrain')
val_df

,model_name,val_acc,top5_val_acc,f1_val,recall,precision
0,videomae_no_augmentations_24,0.797857,0.975714,0.795264,0.797857,0.809697
0,videomae_no_augmentations_14,0.767619,0.968571,0.769840,0.767619,0.791145
0,videomae_no_augmentations_42,0.791161,0.978810,0.790367,0.791161,0.815347
0,videomae_no_augmentations_44,0.801875,0.979286,0.800091,0.801875,0.813962
0,videomae_no_augmentations_26,0.800923,0.971905,0.796945,0.800923,0.817270
0,videomae_no_augmentations_2,0.795208,0.968571,0.794475,0.795208,0.814652
0,videomae_no_augmentations_16,0.806190,0.965952,0.810657,0.806190,0.832871
0,videomae_no_augmentations_79,0.540446,0.903214,0.538985,0.540446,0.622906
0,videomae_no_augmentations_12,0.769018,0.965238,0.768517,0.769018,0.784714
0,videomae_no_augmentations_89,0.801875,0.982857,0.801228,0.801875,0.812395


In [18]:
malta_test_df = pd.read_csv('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/no_pretrain/test_mean_std_results.csv')
malta_test_df

,dictionary,acc_mean,acc_std,top5_acc_mean,top5_acc_std,f1_mean,f1_std,precision_mean,precision_std,recall_mean,recall_std
0,Acessibilidades2,0.061538,0.037157,0.300000,0.074315,0.011276,0.009319,0.007078,0.007055,0.044474,0.018129
1,Acessibilidades3,0.050000,0.017568,0.250000,0.065145,0.014038,0.012728,0.012029,0.016740,0.055263,0.022699
2,Overall,0.048855,0.012043,0.260305,0.021726,0.022406,0.013135,0.020596,0.016746,0.043102,0.009951
3,SpreadTheSign,0.056250,0.062152,0.300000,0.070956,0.026696,0.038570,0.022834,0.034995,0.046507,0.051449
4,UFPE,0.040741,0.047655,0.185185,0.069838,0.010449,0.012375,0.006270,0.007565,0.036364,0.044186
5,UFSC,0.026316,0.027739,0.257895,0.052338,0.002946,0.003131,0.001556,0.001656,0.027778,0.029280
6,UFSC_V2,0.057895,0.016644,0.294737,0.044383,0.009002,0.009309,0.005160,0.006143,0.061111,0.017568


## Augmentations

#### Color Jitter

In [78]:
import os
import re
import pandas as pd
import numpy as np
from collections import defaultdict

def aggregate_metrics(root_dir):
    metrics_dict = defaultdict(lambda: {
        'val': {'acc': [], 'f1': [], 'precision': [], 'recall': []},
        'test': {'acc': [], 'top5_acc': [], 'f1': [], 'precision': [], 'recall': []}
    })

    # 1. Enhanced filename matching
    VAL_REGEX = re.compile(
        r'(val|validation)(_.*)?_results\.csv$', 
        re.IGNORECASE
    )
    TEST_REGEX = re.compile(r'test_metrics_results\.csv$', re.IGNORECASE)

    # 2. Expanded column variants
    COLUMN_VARIANTS = {
        'acc': [
            'acc_minds_val', 'acc_val', 'accuracy_val',
            'val_acc'

        ],
        'f1': [
            'f1_minds_val', 'f1_val', 'f1score_val',
            'f1_val'

        ],
        'precision': [
            'precision_minds_val', 'precision_val',
            'precision'
        ],
        'recall': [
            'recall_minds_val', 'recall_val', 'recall'

        ]
    }

    for dirpath, _, filenames in os.walk(root_dir):
        folder_name = os.path.basename(dirpath)
        if not folder_name.startswith('videomae_kinetics_'):
            continue

        # Parse folder name
        try:
            parts = folder_name.split('_')
            augmentation = '_'.join(parts[2:-2])
            intensity = float(parts[-2])
        except (ValueError, IndexError):
            continue

        # Process files
        for filename in filenames:
            # Match validation/test files
            if VAL_REGEX.search(filename):
                metric_type = 'val'
            elif TEST_REGEX.search(filename):
                metric_type = 'test'
            else:
                continue

            csv_path = os.path.join(dirpath, filename)
            try:
                df = pd.read_csv(csv_path)
                if df.empty:
                    continue

                row = df.iloc[0]
                key = (augmentation, intensity)

                if metric_type == 'val':
                    # Handle column variations
                    collected = {}
                    for metric in ['acc', 'f1', 'precision', 'recall']:
                        for col in COLUMN_VARIANTS[metric]:
                            if col in df.columns:
                                collected[metric] = row[col]
                                break
                        else:
                            print(f"No {metric} column in {csv_path}")
                            break
                    
                    if len(collected) == 4:
                        for m, v in collected.items():
                            metrics_dict[key]['val'][m].append(v)

                else:
                    # Test metrics
                    test_data = {
                        'acc': row['acc_test'],
                        'f1': row['f1_test'],
                        'precision': row['precision_test'],
                        'recall': row['recall_test']
                    }
                    if 'top5_acc_test' in df.columns:
                        test_data['top5_acc'] = row['top5_acc_test']
                    
                    for m, v in test_data.items():
                        metrics_dict[key]['test'][m].append(v)

            except Exception as e:
                print(f"Error in {filename}: {str(e)}")

    # Compile results
    results = []
    for (aug, intensity), data in metrics_dict.items():
        entry = {
            'augmentation': aug,
            'intensity': intensity,
            'num_seeds_val': len(data['val']['acc']),
            'num_seeds_test': len(data['test']['acc'])
        }

        # Validation metrics
        for metric in ['acc', 'f1', 'precision', 'recall']:
            vals = data['val'][metric]
            entry[f'val_{metric}_mean'] = np.nanmean(vals) if vals else np.nan
            entry[f'val_{metric}_std'] = np.nanstd(vals) if vals else np.nan

        # Test metrics
        for metric in ['acc', 'top5_acc', 'f1', 'precision', 'recall']:
            if metric in data['test']:
                vals = data['test'][metric]
                entry[f'test_{metric}_mean'] = np.nanmean(vals) if vals else np.nan
                entry[f'test_{metric}_std'] = np.nanstd(vals) if vals else np.nan

        results.append(entry)

    return pd.DataFrame(results).sort_values(['augmentation', 'intensity'])

In [79]:
cj_results = aggregate_metrics('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter')
cj_results

No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.15_14/slovo_val_metrics_results.csv
No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.45_79/slovo_val_metrics_results.csv
No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.4_89/slovo_val_metrics_results.csv
No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.4_44/slovo_val_metrics_results.csv
No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.4_12/slovo_val_metrics_results.csv
No acc column in /mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/color_jitter/color_jitter/videomae_kinetics_color_jitter_0.35_44/slovo_val_metrics_results

,augmentation,intensity,num_seeds_val,num_seeds_test,val_acc_mean,val_acc_std,val_f1_mean,val_f1_std,val_precision_mean,val_precision_std,...,test_acc_mean,test_acc_std,test_top5_acc_mean,test_top5_acc_std,test_f1_mean,test_f1_std,test_precision_mean,test_precision_std,test_recall_mean,test_recall_std
4,color_jitter,0.05,10,10,0.996575,0.004332,0.996507,0.004410,0.996750,0.004119,...,0.289655,0.015110,0.523448,0.034955,0.220567,0.010342,0.260197,0.015882,0.252437,0.012527
8,color_jitter,0.10,10,10,0.996233,0.003231,0.996168,0.003264,0.996450,0.003066,...,0.288276,0.017392,0.554483,0.022915,0.219057,0.016492,0.253664,0.017438,0.249040,0.014656
0,color_jitter,0.15,10,10,0.998973,0.001569,0.998949,0.001605,0.999042,0.001465,...,0.300690,0.024712,0.550345,0.039353,0.231734,0.019548,0.259585,0.030756,0.257065,0.022203
7,color_jitter,0.20,10,10,0.996575,0.003752,0.996506,0.003810,0.996771,0.003540,...,0.295172,0.019700,0.528966,0.022251,0.233952,0.019953,0.268989,0.040869,0.255923,0.017420
9,color_jitter,0.25,10,10,0.996233,0.003575,0.996158,0.003670,0.996576,0.003234,...,0.304828,0.023448,0.528966,0.025997,0.240404,0.026967,0.272433,0.032464,0.263329,0.020775
6,color_jitter,0.30,10,10,0.998630,0.001678,0.998617,0.001693,0.998708,0.001583,...,0.304828,0.013012,0.547586,0.023326,0.240121,0.016030,0.272723,0.039213,0.262510,0.012359
3,color_jitter,0.35,10,10,0.997603,0.002675,0.997568,0.002721,0.997729,0.002528,...,0.302069,0.022832,0.538621,0.026070,0.237292,0.026261,0.258673,0.030651,0.263484,0.021511
2,color_jitter,0.40,10,10,0.998630,0.001678,0.998593,0.001723,0.998687,0.001608,...,0.309655,0.012884,0.539310,0.025952,0.244829,0.016046,0.261677,0.025073,0.266790,0.012427
1,color_jitter,0.45,10,10,0.976370,0.065221,0.975022,0.069170,0.982470,0.047220,...,0.295862,0.026790,0.542759,0.021156,0.239100,0.026365,0.266303,0.033941,0.256672,0.023257
5,color_jitter,0.50,10,10,0.995890,0.003994,0.995803,0.004078,0.996036,0.003836,...,0.310345,0.027929,0.532414,0.039952,0.240837,0.025491,0.262663,0.024898,0.267135,0.024699


#### Random Rotation

In [80]:
rr_results = aggregate_metrics('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/random_rotation')
rr_results

,augmentation,intensity,num_seeds_val,num_seeds_test,val_acc_mean,val_acc_std,val_f1_mean,val_f1_std,val_precision_mean,val_precision_std,...,test_acc_mean,test_acc_std,test_top5_acc_mean,test_top5_acc_std,test_f1_mean,test_f1_std,test_precision_mean,test_precision_std,test_recall_mean,test_recall_std
6,random_rotation,5.0,10,10,0.998214,0.003293,0.998263,0.003206,0.998474,0.002782,...,0.297241,0.035091,0.527586,0.010345,0.229501,0.026965,0.264347,0.027749,0.257567,0.029953
7,random_rotation,10.0,10,10,0.999333,0.001333,0.999321,0.001357,0.999354,0.001292,...,0.307586,0.018037,0.510345,0.006897,0.235027,0.013651,0.270274,0.023638,0.267760,0.017852
2,random_rotation,15.0,10,10,0.996905,0.002383,0.996879,0.002419,0.997063,0.002296,...,0.301379,0.021156,0.517241,0.034483,0.229797,0.019785,0.263814,0.026733,0.260171,0.018975
1,random_rotation,20.0,10,10,0.997238,0.002650,0.997188,0.002668,0.997375,0.002430,...,0.293103,0.019323,0.517241,0.027586,0.224967,0.013043,0.264453,0.026183,0.252639,0.016723
3,random_rotation,25.0,10,10,0.998667,0.001633,0.998621,0.001689,0.998667,0.001633,...,0.291724,0.023499,0.524138,0.027586,0.227063,0.023552,0.265169,0.035594,0.255194,0.020828
8,random_rotation,30.0,10,10,0.998667,0.002211,0.998621,0.002287,0.998667,0.002211,...,0.316552,0.013947,0.544828,0.027586,0.242815,0.010431,0.265852,0.024167,0.275006,0.013182
4,random_rotation,35.0,10,10,0.996595,0.002615,0.996558,0.002659,0.996771,0.002504,...,0.310345,0.016893,0.544828,0.013793,0.242511,0.020882,0.279047,0.042152,0.269264,0.015292
9,random_rotation,40.0,10,10,0.997571,0.003497,0.997554,0.003523,0.997729,0.003250,...,0.320690,0.021424,0.531034,0.020690,0.243041,0.023821,0.255579,0.027026,0.276754,0.016378
0,random_rotation,45.0,10,10,0.997905,0.003554,0.997912,0.003536,0.998011,0.003394,...,0.315862,0.014066,0.524138,0.006897,0.237357,0.015027,0.264977,0.026697,0.271956,0.014270
5,random_rotation,50.0,10,10,0.997929,0.002328,0.997915,0.002320,0.998042,0.002155,...,0.312414,0.028444,0.533333,0.060735,0.236878,0.022569,0.262116,0.033782,0.270270,0.023646


#### Random Perspective

In [81]:
rp_results = aggregate_metrics('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/random_perspective')
rp_results

,augmentation,intensity,num_seeds_val,num_seeds_test,val_acc_mean,val_acc_std,val_f1_mean,val_f1_std,val_precision_mean,val_precision_std,...,test_acc_mean,test_acc_std,test_top5_acc_mean,test_top5_acc_std,test_f1_mean,test_f1_std,test_precision_mean,test_precision_std,test_recall_mean,test_recall_std
4,random_perspective,0.05,10,10,0.996918,0.004180,0.996864,0.004235,0.997078,0.003939,...,0.287586,0.023296,NaN,NaN,0.221983,0.018296,0.260704,0.032998,0.248726,0.020338
0,random_perspective,0.10,10,10,0.997260,0.002563,0.997197,0.002650,0.997417,0.002390,...,0.278621,0.023731,NaN,NaN,0.223681,0.019199,0.274151,0.043577,0.243228,0.021638
5,random_perspective,0.15,10,10,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.276552,0.020563,NaN,NaN,0.207843,0.017873,0.249464,0.028331,0.236046,0.015596
3,random_perspective,0.20,10,10,0.998630,0.002272,0.998612,0.002307,0.998766,0.002006,...,0.288276,0.034317,NaN,NaN,0.223547,0.024997,0.259781,0.018118,0.248419,0.028588
7,random_perspective,0.25,10,10,0.998630,0.002272,0.998618,0.002273,0.998729,0.002093,...,0.283448,0.020330,NaN,NaN,0.219328,0.020762,0.254532,0.032690,0.245663,0.017205
1,random_perspective,0.30,10,10,0.999315,0.001370,0.999307,0.001386,0.999375,0.001250,...,0.286207,0.018312,NaN,NaN,0.222559,0.011362,0.253563,0.025924,0.245681,0.015057
2,random_perspective,0.35,10,10,0.997603,0.003082,0.997543,0.003181,0.997729,0.002887,...,0.275862,0.027413,NaN,NaN,0.209685,0.023366,0.243984,0.023935,0.236530,0.022875
9,random_perspective,0.40,10,10,0.999315,0.001370,0.999310,0.001379,0.999333,0.001333,...,0.285517,0.014201,NaN,NaN,0.216536,0.014084,0.253723,0.026632,0.246361,0.013940
6,random_perspective,0.45,10,10,0.998630,0.001678,0.998608,0.001705,0.998667,0.001633,...,0.283448,0.027143,NaN,NaN,0.213467,0.026292,0.248226,0.032746,0.247671,0.019335
8,random_perspective,0.50,10,10,0.999315,0.001370,0.999310,0.001379,0.999333,0.001333,...,0.293103,0.020048,NaN,NaN,0.228367,0.018582,0.250805,0.031404,0.253901,0.015477


#### Mixup

In [82]:
mu_results = aggregate_metrics('/mnt/G-SSD/BRACIS/BRACIS-2024/lightning_logs/minds/augs/mixup')
mu_results

,augmentation,intensity,num_seeds_val,num_seeds_test,val_acc_mean,val_acc_std,val_f1_mean,val_f1_std,val_precision_mean,val_precision_std,...,test_acc_mean,test_acc_std,test_top5_acc_mean,test_top5_acc_std,test_f1_mean,test_f1_std,test_precision_mean,test_precision_std,test_recall_mean,test_recall_std
17,mixup,0.05,10,10,0.998288,0.003509,0.998229,0.003641,0.998511,0.003059,...,0.286207,0.024914,NaN,NaN,0.223753,0.022860,0.258090,0.030450,0.247891,0.020842
11,mixup,0.10,10,10,0.998288,0.002297,0.998261,0.002327,0.998354,0.002190,...,0.291034,0.033475,NaN,NaN,0.225920,0.035192,0.250991,0.041352,0.248937,0.029314
12,mixup,0.15,10,10,0.998973,0.001569,0.998964,0.001583,0.999021,0.001497,...,0.279310,0.017515,NaN,NaN,0.211180,0.020508,0.253710,0.036044,0.239075,0.016976
13,mixup,0.20,10,10,0.998630,0.002740,0.998596,0.002809,0.998787,0.002428,...,0.288966,0.036421,NaN,NaN,0.224725,0.028471,0.266163,0.033214,0.246466,0.033668
15,mixup,0.25,10,10,0.998973,0.001569,0.998951,0.001603,0.999021,0.001497,...,0.297931,0.025768,NaN,NaN,0.222666,0.026386,0.263285,0.036592,0.256540,0.023774
14,mixup,0.30,10,10,0.998288,0.003157,0.998243,0.003241,0.998500,0.002630,...,0.293103,0.020517,NaN,NaN,0.225388,0.018936,0.259907,0.020063,0.248474,0.020258
10,mixup,0.35,10,10,0.955137,0.132313,0.953107,0.138345,0.966820,0.097326,...,0.254483,0.040148,NaN,NaN,0.195984,0.023737,0.238253,0.027405,0.220814,0.030931
18,mixup,0.40,10,10,0.998973,0.001569,0.998949,0.001605,0.999042,0.001465,...,0.286897,0.045034,NaN,NaN,0.216986,0.045716,0.252249,0.052908,0.245708,0.041071
19,mixup,0.45,10,10,0.999658,0.001027,0.999654,0.001039,0.999687,0.000938,...,0.276552,0.029333,NaN,NaN,0.209014,0.031577,0.245982,0.044879,0.235921,0.025675
16,mixup,0.50,10,10,0.998288,0.002297,0.998257,0.002348,0.998393,0.002198,...,0.273103,0.039642,NaN,NaN,0.206370,0.042614,0.252732,0.058439,0.231014,0.041852
